In [ ]:
/*** ONE TIME ONLY ***/
use role accountadmin;
use database customer_support;
use schema support;
use warehouse COMPUTE_WH;


-- CREATE OR REPLACE CORTEX SEARCH SERVICE customer_master_data_search_svc
--   ON full_details
--   WAREHOUSE = COMPUTE_WH
--   TARGET_LAG = '1 day'
--   AS (
--    select full_details
--    from customer_master_data
-- );




In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
select * from customer_support.support.customer_master_data limit 10;

In [ ]:
select count(1) from customer_support.support.customer_master_data;

In [ ]:
from snowflake.core import Root

root = Root(session)

# query service
svc = (root
  .databases["customer_support"]
  .schemas["support"]
  .cortex_search_services["customer_master_data_search_svc"]
)



In [ ]:
resp = svc.search(
  query="Patton 887 Aaron Center   WY 15339",
  columns=["full_details"],
  limit=1
).to_json()

st.info(resp)

In [ ]:
select * from customer_support.support.new_customer_data limit 10;

In [ ]:
import json
new_data = session.table("customer_support.support.new_customer_data").select("FULL_DETAILS").to_pandas()
df = pd.DataFrame(columns=['New Address', 'Matching Existing address'])
for index, row in new_data.iterrows():
    resp = svc.search(
        query=row["FULL_DETAILS"],
        columns=["full_details"],
        limit=1
    ).to_json()
    j = json.loads(resp)
    df.loc[index] = [ row["FULL_DETAILS"], j["results"][0]["full_details"]]
st.write(df)

## Use VECTOR EMBEDDING to see how close out matches are

In [ ]:
drop table if exists "compared_data";

In [ ]:
session.write_pandas(df,'compared_data',overwrite=True, table_type="temp") 

In [ ]:
select * from "compared_data" limit 100;

In [ ]:
with compare_data_vector 
as
(
select 
*,
snowflake.cortex.embed_text('e5-base-v2', "New Address") as new_addr_vector, 
snowflake.cortex.embed_text('e5-base-v2', "Matching Existing address") as matching_addr_vector
from "compared_data"
)
SELECT
   "New Address",
   "Matching Existing address",
   VECTOR_COSINE_SIMILARITY(new_addr_vector, matching_addr_vector) AS score
FROM 
    compare_data_vector
;

In [ ]:
select snowflake.cortex.complete('mistral-7b', 'How close these two given addresses. 
<address_1> ' || "New Address" || '</address_1>
<address_2> ' || "Matching Existing address" || '</address_2>
Provide a matching score of 1 to 100, where 100 being identical and 1 no matches at all. Just provide the score, no other verbiage') as MATCH_DETAILS,
"New Address",
"Matching Existing address"
from "compared_data"


## Find similar customer based on orders placed

In [ ]:
select * from DEMO_DB.CRM.orders

In [ ]:
--let's find the best matching customers
SELECT
   v.cust_id cust_id,
   v.agg products_ordered,
   m.cust_id best_matching_customer,
   m.agg matching_products,
   VECTOR_COSINE_SIMILARITY(v.agg_vector, m.agg_vector) AS match_score
FROM 
    DEMO_DB.CRM.orders_vector v inner join
    DEMO_DB.CRM.orders_vector m on v.cust_id != m.cust_id
QUALIFY RANK() OVER(PARTITION BY v.cust_id ORDER BY match_score desc) = 1
order by v.cust_id

In [ ]:
SELECT
   v.cust_id cust_id,
   v.agg products_ordered,
   m.cust_id best_matching_customer,
   m.agg matching_products,
   array_except(STRTOK_TO_ARRAY(m.agg,','),STRTOK_TO_ARRAY(v.agg,',')) UPSELL_OPP,
   VECTOR_COSINE_SIMILARITY(v.agg_vector, m.agg_vector) AS match_score
FROM 
    DEMO_DB.CRM.orders_vector v inner join
    DEMO_DB.CRM.orders_vector m on v.cust_id != m.cust_id
QUALIFY RANK() OVER(PARTITION BY v.cust_id ORDER BY match_score desc) = 1
order by v.cust_id